# [LELEC2870] - Machine Learning

## Practical session 6 - PCA and ICA

Prof. M. Verleysen<br>
Prof. J. Lee<br>

**Teaching assistants :**  
Edouard Couplet : edouard.couplet@uclouvain.be  
Seyed Ghasemzadeh : seyed.ghasemzadeh@uclouvain.be  
Niels Sayez : niels.sayez@uclouvain.be  
Mathieu Simon : mathieu.simon@uclouvain.be  
Antoine Vanderschueren : antoine.vanderschueren@uclouvain.be

## 1) Getting started

The aim of this exercise session is to get familiar with the source separation methods and more precisely, with principal component analysis (PCA) and independent component analysis (ICA). (Remark: for those of you who have followed it, it is the same intro as LGBIO2020 but don't worry we will do different things during the exercise session ;) )

In a source separation problem represented in the figure below, it is assumed that we observe a set of $n$ random variables (denoted by $X \in \mathbb{R}^n$) resulting from the linear mixing of other *unknown* source variables ($S\in \mathbb{R}^m$). In other words, if we denote the mixing matrix by $A \in \mathbb{R}^{n \times m}$, the generative model

\begin{equation}
X = A \cdot S
\label{eq:source_sepa_model}
\end{equation}
is considered. We will limit ourselves to the cases where $m = n$.

<img src="data/BSS.png" width = "500">

<br>

In this setting, both methods (PCA and ICA) aim to find the so-called unmixing matrix $W\approx A^{-1} \in \mathbb{R}^{n \times n}$ allowing to characterize the sources from the observed variables:

\begin{equation}
S \approx W\cdot X
\label{eq:unmixing_model}
\end{equation}

<br>

In practice, one will use several **observations** of the random variable $X$ to determine the unmixing matrix in both cases, as you will see during the lecture. In order to obtain this unmixing matrix, PCA assumes that the sources are **uncorrelated** and ICA assumes that the sources are **independent**.


<table style="width:60%">
            <thead>
                <tr>
                    <th> PCA </th>
                    <th> ICA </th>
                </tr>
            </thead>
            <tbody>
                <tr>
                    <td> Find a transformation that DECORRELATES the variables </td>
                    <td> Find a transformation that makes variables as INDEPENDANT as possible </td>
                </tr>
                <tr>
                    <td> $ E[XY] ~=~ E[X]~E[Y] $ </td>
                    <td> $ E[f(X)~g(Y)] ~=~ E[f(X)]~E[g(Y)] $ </td>
                </tr>
                <tr>
                    <td> Correlation measures analyze the existence of a linear relation between variables </td>
                    <td> Dependence measures analyze the existence of any relation between variables </td>
                </tr>
                <tr>
                    <td> <img src="data/PCA.png" width = "200"> </td>
                    <td> <img src="data/ICA.png" width = "200"> </td>
                </tr>
            </tbody>
    </table>      

In [ ]:
from matplotlib import pyplot as plt
try:
    import seaborn as sns
    use_seaborn = True
    sns.set()
except:
    use_seaborn = False
import numpy as np
from scipy.io import wavfile as wf
import IPython.display as ipd
from sklearn.decomposition import FastICA, PCA
import math
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import scipy.io

<div class="alert alert-success">
<b>Generate the dataset</b>  <br>
First, we ask you to generate a small 2D dataset. <li> Generate two source signals $\{s_1, s_2\}$ of 2000 samples, following a uniform distribution:<br>

\begin{equation}
p(s_i) =
     \begin{cases}
        \frac{1}{2\sqrt{3}} & \text{if } |s_i| \leq \sqrt{3} \\
        0 & \text{otherwise}
     \end{cases}
\end{equation}<br>
<li> Generate the mixed signals $X=A\cdot S$ using the mixing matrix $\mathbf{A}$ defined as:<br>
       
\begin{equation}
\mathbf{A} = \left(
\begin{array}{cc}
 2 & 3 \\
 2 & 1 
\end{array}\right)
\end{equation}<br>

</div>

In [ ]:
# Generate the sources
s1 = #TO DO
s2 = #TO DO
s = #TO DO

# Generate the mixing matrix
A = #TO DO
X = #TO DO

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].scatter(s[0],s[1]);
axes[0].set_title('original data',fontsize=16)
axes[1].scatter(X[0],X[1]);
axes[1].set_title('Mixture', fontsize=16)
fig.tight_layout()

### 1.1) PCA

The objective of Principal Component Analysis (PCA) is to find the matrix $W$ that allows to obtain uncorrelated white variables $Y$ from the correlated ones $X$.

$$Y = W\cdot X$$

During the lectures, it has been demonstrated that the matrix $W$ that can achieve this is given by : 

$$W = \Lambda^{-1/2}V^{T}$$ 

where $\Lambda$ and $V$ are respectively the diagonal matrix of eigenvalues and the matrix of eigenvectors of the covariance matrix $\frac{1}{N}XX^{T}$. 

As a result the covariance of the output variables $Y$ is given by : $\frac{1}{N}YY^{T}= \frac{1}{N}(\Lambda^{-1/2}V^{T}X)(\Lambda^{-1/2}V^{T}X)^{T}=I \quad \rightarrow$  **white output !**

<div class="alert alert-success">
<b>PCA implementation</b>  <br>
Complete the MyPCA class below in order to implement the PCA whitening using singular value decomposition (SVD) np.linalg.svd of the variables X.
</div>

<div class="alert alert-warning">
<b>Question</b>  <br>
Notice that before applying PCA the data are normalized. Why do we need to center our data ? How would it affect the equations written above if the data are not centered ? 
</div>

In [ ]:
class MyStandardScaler():
    def __init__(self, with_std=True):
        self.with_std = with_std
    
    def fit(self, X, y=None):
        ''' Computes the mean and standard deviation (if with_std is True) based on X '''
        self.mean = np.mean(X, axis=1)
        if self.with_std:
            X = X - (np.ones(X.T.shape)*self.mean).T
            self.std = np.sqrt(np.var(X, axis=1))
        return self

    def transform(self, X):
        ''' Transforms X based on the previously computed mean and std and returns it '''
        X = X - (np.ones(X.T.shape)*self.mean).T
        if self.with_std:
            X = X / (np.ones(X.T.shape)*self.std).T
        return X
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)


class MyPCA():
    def __init__(self, n_components, ):
        self.n_comps = n_components
        
    def fit(self, X, y=None):
        ''' Computes the transformation matrix W based on the SVD of X '''
        u,s,vh = #TO DO
        self.w = #TO DO
        return self
    
    def transform(self, X):
        ''' Whiten the data based on the previously computed W matrix '''
        return #TO DO
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

pca = Pipeline([('scaler', MyStandardScaler()), ('pca', MyPCA(n_components=2))])

<div class="alert alert-success">
<b>PCA implementation</b>  <br>
Use your PCA implementation in order fit and transform the correlated variables X. Then, compute the covariance matrices for S, X and Y. What do you observe ? Are the output variables whiten ? Are they independant ?
</div>

In [ ]:
Y = #TO DO

plt.figure(figsize=(6,6))
plt.title('PCA')
plt.scatter(Y[0],Y[1])
plt.show();

print('Covariance matrix source : \n', #TODO) 
print('Covariance matrix mixture : \n', #TODO) 
print('Covariance matrix after PCA : \n', #TODO) 

### 1.2) ICA

The objective of Independent Component Analysis (ICA) is to find the matrix $W$ that allows to recover the independant sources $S$ from the data $X$. 

$$S \approx W\cdot X$$

The ICA algorithm makes use of a measure of how statistically independent the sources are, instead of using variance. This measure is usually based on nonGaussianity. Indeed, according to the central limit theorem, the PDF of a sum of n independent random variables tends to be gaussian. Therefore the objective is to find a $W$ such that the output PDFs are as different as possible from a Gaussian.

To characterise the nonGaussianity, here we use negentropy as a measure of distance to normality. The negentropy is defined as

$$J(x) = h(x_{G})-h(x)$$ 

where $h(x)$ is the differential entropy of the data x and $h(x_{G})$ is the differential entropy of the gaussian with the same mean and variance as x. The negentropy is always positive (since the differential entropy is maximum in the gaussian case i.e. for $X_G$) and is zero if x is gaussian. 

The objective therefore becomes : find the $W$ that maximizes $J(Y)=J(W\cdot X)$

In practice, the negentropy needs to be estimated and the optimization problem to solve goes far beyond the scope of this course. Thus, we directly provide you the implementation below ;)

<div class="alert alert-warning">
<b>Question</b>  <br>
Observe the fit function of the MyICA class (you don't have to understand the other parts of the code !). What do you notice ? Why is the ICA class extending the PCA one ? 
</div>

NB : This is only one way to proceed and they are other possibilities. For those of you who are interested do not hesitate to have a look at the ICA paper : https://pubmed.ncbi.nlm.nih.gov/10946390/

In [ ]:
class MyICA(MyPCA):
    def __init__(self, n_components):
        super().__init__(n_components)
        
    def fit(self, X, y=None):
        # Make the PCA transformation to whiten the data before making ICA
        super().fit(X) 
        X_whiten = super().transform(X)
        # Do the ICA fit
        V = self.ica_fit(X_whiten) 
        self.w = V @ self.w
        return self
    
    def g(self, x):
        return x*np.exp(-(x ** 2) / 2)

    def dg(self, x):
        return (1 - x ** 2)*(np.exp(-(x ** 2) / 2))

    def calc_w_hat(self, W, X):
        w_hat = (X * self.g(W.T @ X)).mean(axis=1) - self.dg(W.T @ X).mean() * W
        w_hat /= np.sqrt((w_hat ** 2).sum())
        return w_hat

    def ica_fit(self, X, iterations=100, tolerance=1e-5):
            num_components = X.shape[0]
            W = np.zeros((num_components, num_components), dtype=X.dtype)
            distances = {i: [] for i in range(num_components)}
    
            for i in np.arange(num_components):
                w = np.random.rand(num_components)
                for j in np.arange(iterations):
                    w_new = self.calc_w_hat(w, X)
                    if(i >= 1):
                        w_new -= np.dot(np.dot(w_new, W[:i].T), W[:i])
                    distance = np.abs(np.abs((w * w_new).sum()) - 1)
                    w = w_new
                    if(distance < tolerance):
                        break;
                    distances[i].append(distance)
                W[i, :] = w
            return W
        
ica = Pipeline([('scaler', MyStandardScaler()), ('ica', MyICA(n_components=2))])

<div class="alert alert-success">
<b>ICA implementation</b>  <br>
Use the ICA implementation in order to recover the independant sources S. What do you observe ? 
</div>

In [ ]:
S_recover = #TO DO

plt.figure(figsize=(6,6))
plt.title('ICA')
plt.scatter(S_recover[0],S_recover[1])
plt.show();

### 1.3) Gaussian case

<div class="alert alert-success">
<b>Generate gaussian data</b>  <br>
Generate a 2D dataset of 2000 samples following a normal distribution and apply on it the same mixing matrix as in the previous case. 
</div>

In [ ]:
# Generate the sources
s1 = #TO DO
s2 = #TO DO
s = #TO DO

# Generate the mixing matrix
A = #TO DO
X = #TO DO

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].scatter(s[0],s[1]);
axes[0].set_title('original data',fontsize=16)
axes[1].scatter(X[0],X[1]);
axes[1].set_title('Mixture', fontsize=16)
fig.tight_layout()

<div class="alert alert-success">
<b>Gaussian case</b>  <br>
Use ICA to recover the sources S. What do you notice ? Can you explain what you observe ? How is this problem related to ICA ? 
</div>

In [ ]:
S_recover = #TO DO

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].scatter(s[0],s[1]);
axes[0].set_title('original data',fontsize=16)
axes[1].scatter(S_recover[0],S_recover[1])
axes[1].set_title('ICA', fontsize=16)
fig.tight_layout()

**Remark**

Here for PCA and ICA, we have made custom implementations but in practice we can use ready-made functions. In the remaining parts of this exercise session, you are asked to use sklearn.decomposition.PCA and sklearn.decomposition.FastICA to perform the PCA and ICA decompositions.

## 2) PCA application : dimensionality reduction

Sometimes it is useful to reduce the dimension of the data as common datasets are made of a large number of features (i.e. high dimensionality) which causes problems for the models because :
   - Training time increases exponentially with number of features
   - The curse of dimensionality
   
<div class="alert alert-warning">
<b>Question</b>  <br>
What is the curse of dimensionality ? 
</div>
   
PCA offers a logical framework for dimensionality reduction. Indeed PCA tries to find the axes with maximum variance which is equivalent to find the axes that minimize the projection error. 

<img src="data/minmax.png" width = "600">

In [ ]:
# dataset we will use for dimensionality reduction
df = scipy.io.loadmat(f"data/diabetes.mat")
X = df["X"]
X_names = ["age", "sex", "bmi", "blood_pressure", "serum_1","serum_2", "serum_3", "serum_4", "serum_5", "serum_6"]
n_samples, n_feats = X.shape
X[:,0]=X[:,0]*10 #for educational purpose when discussing the scale ;)
t = df["t"]
t_names = [" blood sugar level"]

<div class="alert alert-success">
<b>PCA dimensionality reduction</b>  <br>
Use sklearn.decomposition.PCA in order to extract the PCA components. Plot the amount of explained variance in function of the number of kept features. What does this graph represent ? Can you explain it based on the eigenvalues used by PCA ? How many PCA components would you keep ? 

hint : Start by answering the following question : What does the eigenvalues and the eigenvectors represent in PCA ? 
</div>

In [ ]:
# Normalize the data !!
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)

# TO COMPLETE, do the PCA decomposition of x_scaled and plot the the amount of explained variance 
# in function of the number of kept features

<div class="alert alert-success">
<b>Normalization</b>  <br>
Notice that before making the PCA decomposition for dimensionality reduction the data is normalized. Why do we need to scale the data ? Using sklearn.decomposition.PCA plot the PCA eigenvalues for x and x_scaled. What do you notice ? How does it affect our dimensionality reduction method ?
</div>

In [ ]:
# For scaled features
var_scaled = #TO DO

# For raw features
# TO DO
var_notscaled = #TO DO

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].bar(np.arange(1,11),var_scaled);
axes[0].set_title("Scaled features PCA eigenvalues",fontsize=16)
axes[1].bar(np.arange(1,11),var_notscaled);
axes[1].set_title("Raw features PCA eigenvalues", fontsize=16)
fig.tight_layout()

## 3) ICA application : the cocktail party

We will now try to use ICA to solve the classical cocktail party problem. The objective is to be able to isolate the different sources in an audio recording. 

In [ ]:
# Open the audio files
print("\033[4mSource 1\033[0m : talking man")
r1, s1 = wf.read("data/talk.wav")
ipd.display(ipd.Audio(s1,rate=r1))

print("\033[4mSource 2\033[0m : music")
r2, s2 = wf.read("data/music.wav")
ipd.display(ipd.Audio(s2,rate=r2))

print("\033[4mSource 3\033[0m : wind")
r3, s3 = wf.read("data/wind.wav")
ipd.display(ipd.Audio(s3,rate=r3))

<div class="alert alert-success">
<b>Mixture</b>  <br>
Write the function mix_sources that makes a linear mixing of the different independent sources. 
</div>

In [ ]:
def mix_sources(sources, noise=False, noiselevel= 0.01):
    
    # scale all audios to same duration and scale their intensities
    minSize = np.min([len(j) for j in sources])
    for i in range(len(sources)):
        sources[i] = (sources[i] / (np.max(sources[i]) / 2) - 0.5)[:minSize]
        
    # make the mixture
    #TO DO
    
    # add gaussian noise
    if(noise):
        #TO DO
        
    return #TO DO

# Lets test our mixture
x = mix_sources([s1, s2, s3], True)
print("\033[4mMixed sources\033[0m")
ipd.display(ipd.Audio(x,rate=r1))

<div class="alert alert-success">
<b>ICA</b>  <br>
Use sklearn.decomposition.FastICA to recover the audio sources.
</div>

<div class="alert alert-warning">
<b>question</b>  <br>
Is the recovered source 1 corresponding to the original source 1 ? Is the recovered source 2 corresponding to the original source 2 ? Can you explain why ? What are the two indeterminacies of ICA ?
</div>

<div class="alert alert-warning">
<b>question</b>  <br>
Is the noise separated from the other sources ? can you explain why ? 
</div>

In [ ]:
# TO DO
output = #TO DO

In [ ]:
print("\033[4mRecovered source 1\033[0m")
ipd.display(ipd.Audio(output[:,0],rate=r1))
print("\033[4mRecovered source 2\033[0m")
ipd.display(ipd.Audio(output[:,1],rate=r1))
print("\033[4mRecovered source 3\033[0m")
ipd.display(ipd.Audio(output[:,2],rate=r1))